## Getting data from Google Search console into BigQuery

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import requests
import json
import pandas as pd
from google.cloud import bigquery

### Authenticate
For authenttication we should create a service account and grant this service account access to the Google Search Console properties that we want to access.

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/webmasters']
SERVICE_ACCOUNT_FILE = 'my_key.json'
credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

In [ ]:
service = build(
    'webmasters',
    'v3',
    credentials=credentials
)

### Parameters
Define the parameters for the query.

In [ ]:
site_url = "https://www.mandmdirect.com"
PROPERTIES = ["https://www.google.com","https://www.apple.com"]
start_date = "2020-01-01"
end_date = "2020-01-01"
BQ_DATASET_NAME = 'test'
BQ_TABLE_NAME = 'test_sc'

In [ ]:
request = {
      'startDate': start_date,
      'endDate': end_date,
      'dimensions': ["page","device","query"], # uneditable to enforce a nice clean dataframe at the end!
      'rowLimit': 25000,
      'startRow': 0
       }

### Make the query and get the data

In [ ]:
# Make the request to grab the data
response = service.searchanalytics().query(siteUrl=site_url, body=request).execute()
# Go down one level in the response object
res = response['rows']
# Create a DataFrame of the results
df = pd.DataFrame.from_dict(res)

In [ ]:
df

### Clean up the data
Clean and format the data into a nice neat Pandas DataFrame.

In [ ]:
# By default the keys/dimensions are in a single column, let's split them out into separate columns.
new_cols = df['keys'].astype(str).str.replace("[","").str.replace("]","")
new_cols = new_cols.str.split(pat=',',expand=True,n=2)

# Give the columsn sensible names
new_cols.columns = ["page","device","keyword"]

# Get rid of quotation marks
new_cols['device'] = new_cols['device'].str.replace("'","").str.lower()
new_cols['keyword'] = new_cols['keyword'].str.replace("'","")
new_cols['page'] = new_cols['page'].str.replace("'","")

# Bring back a key from the intial dataframe so we can join
new_cols['key'] = df['keys']

# Join in the new clean columns to our intiial data
result = pd.concat([new_cols,df], axis=1, join='inner')

# Drop the key columns
result = result.drop(["key","keys"],axis=1)

In [ ]:
result.head()

### Define a reusable function to do this for us

In [ ]:
def get_sc_df(site_url,start_date,end_date,start_row):
    """Grab Search Console data for the specific property and send it to BigQuery."""
    
    request = {
      'startDate': start_date,
      'endDate': end_date,
      'dimensions': ["page","device","query"], # uneditable to enforce a nice clean dataframe at the end!
      'rowLimit': 25000,
      'startRow': start_row
       }
    
    response = service.searchanalytics().query(siteUrl=site_url, body=request).execute()
    
    if len(response) > 1:
    
        x = response['rows']

        df = pd.DataFrame.from_dict(x)

        # By default the keys/dimensions are in a single column, let's split them out into separate columns.
        new_cols = df['keys'].astype(str).str.replace("[","").str.replace("]","")
        new_cols = new_cols.str.split(pat=',',expand=True,n=2)

        # Give the columsn sensible names
        new_cols.columns = ["page","device","keyword"]

        # Get rid of quotation marks
        new_cols['device'] = new_cols['device'].str.replace("'","").str.lower().str.trim()
        new_cols['keyword'] = new_cols['keyword'].str.replace("'","")str.trim()
        new_cols['page'] = new_cols['page'].str.replace("'","")str.trim()

        # Bring back a key from the intial dataframe so we can join
        new_cols['key'] = df['keys']

        # Join in the new clean columns to our intiial data
        result = pd.concat([new_cols,df], axis=1, join='inner')

        # Drop the key columns
        result = result.drop(["key","keys"],axis=1)
        
        # Add a website identifier
        result['website'] = site_url
        
        # establish a BigQuery client
        client = bigquery.Client.from_service_account_json(SERVICE_ACCOUNT_FILE)
        dataset_id = BQ_DATASET_NAME
        table_name = BQ_TABLE_NAME
        # create a job config
        job_config = bigquery.LoadJobConfig()
        # Set the destination table
        table_ref = client.dataset(dataset_id).table(table_name)
        job_config.destination = table_ref
        job_config.write_disposition = 'WRITE_APPEND'

        load_job = client.load_table_from_dataframe(result, table_ref, job_config=job_config)
        load_job.result()
    
        return result
    else:
        print("There are no more results to return.")

In [ ]:
y = get_sc_df("https://www.mandmdirect.com","2020-01-01","2020-01-01",100000)

In [ ]:
y

### Look through multiple pages of results

In [ ]:
for x in range(0,100000,25000):
    y = get_sc_df("https://www.mandmdirect.com","2020-01-01","2020-01-01",x)
    y.info()
    if len(y) < 25000:
        break
    else:
        continue

### Loop over multiple properties

In [ ]:
for p in PROPERTIES:
    for x in range(0,100000,25000):
        y = get_sc_df(p,"2020-01-01","2020-01-01",x)
        if len(y) < 25000:
            break
        else:
            continue